# <div align="center" style="color: #ff5733;">Sil Account level tagging based on Item Purchased</div>

# Logic for this is to select the data based on the Item purchased. If the Item Purchased in Mobile phone then it is a Mobile store

Created by : Dwaipayan Chakroborti<br>
Requestor: Anastasia -- Refer to her group chat with Heaven, Biswa, Me and Sreekanth<br>
Dated: 2024-09-30

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

In [2]:
sq = """     
SELECT
  lmt.digitalLoanAccountId,
  lmt.loanAccountNumber,
  lmt.purpleKey,
  lmt.merchantName,
  lmt.startApplyDateTime,
  lmt.applicationStatus,
  lmt.decision_date,
  lmt.new_loan_type,
  lmt.flagDisbursement,
  lmt.disbursementDateTime,
  lmt.disbursedLoanAmount,
  lmt.downPaymentAmount,
  tbtc.loanCategoryId,
  tbtc.createdDate,
  tbtc.brandName,
  tbtc.skuNo,
  tbtc.price,
  tbpm.product_id,
  tbpm.product_category,
  tbpm.product_description,
  asf.STORE_TAG,    --- This is the tag maintained by SIL team
  tbpm.product_downpayment,
  tbpm.product_crifcode,
FROM
  `risk_credit_mis.loan_master_table` lmt
LEFT JOIN
  prj-prod-dataplatform.dl_loans_db_raw.tdbk_bnpl_trans_category tbtc
ON
  lmt.digitalLoanAccountid = tbtc.digitalLoanAccountid
LEFT JOIN
  prj-prod-dataplatform.dl_loans_db_raw.tdbk_bnpl_product_mtb tbpm
ON
  tbpm.product_id = tbtc.loanCategoryId
LEFT JOIN
  `prj-prod-dataplatform.worktable_datachampions.AREA_SPLIT_FINAL` asf
ON
  UPPER(asf.PURPLE_KEY) = UPPER(lmt.purpleKey) --
 WHERE
 upper(lmt.new_loan_type) like '%SIL%'
 and lmt.flagDisbursement = 1
 and lmt.disbursementDateTime is not null
  ;
"""

In [ ]:
df = client.query(sq).to_dataframe(progress_bar_type='tqdm')

In [6]:
df.to_csv(r"LoanLevelPurchasedITEM.csv", index = False)

# New query with the code part from Marek also included for better clarity

In [6]:
sq = """SELECT
  lmt.digitalLoanAccountId,
  lmt.loanAccountNumber,
  lmt.purpleKey,
  lmt.merchantName,
  lmt.startApplyDateTime,
  lmt.applicationStatus,
  lmt.decision_date,
  substr(cast(date_trunc(lmt.decision_date, MONTH) as string),0,10) as mth,
  lmt.employmentstatus,
  case
  when EXTRACT(HOUR FROM lmt.termsAndConditionsSubmitDateTime)<18 then "<18"
  else "18+" end as application_hour,
  case
  when upper(lmt.purposeDescription) like "%MOBILE%" then "mobile"
  else "non-mobile" end as goods_type,
  case
  when c.store_type=1 then "appliance"
  else "mobile" end as pos_type,
  c.mer_name as retailer,
  lmt.new_loan_type,
  lmt.loantype,
  lmt.flagApproval as approved,
  lmt.flagDisbursement as disbursed,
  lmt.flagrejection as rejected,
  lmt.disbursedloanamount as volume_out,
  lmt.disbursedLoanTenur as tenor,
  lmt.disbursementDateTime,
  lmt.downPaymentAmount,
  case
    when lmt.loanRequestAmount<=10000 then "<10k"
    when lmt.loanRequestAmount<=15000 then "<15k"
    when lmt.loanRequestAmount<=20000 then "<20k"
    when lmt.loanRequestAmount<=25000 then "<25k"
    when lmt.loanRequestAmount<=30000 then "<30k"
    else "30k+" end as ca_category,
  case
  when safe_cast(lmt.downPaymentAmount as float64)/(safe_cast(lmt.downPaymentAmount as float64)+lmt.loanrequestamount)<0.195 then "<20%"
  when safe_cast(lmt.downPaymentAmount as float64)/(safe_cast(lmt.downPaymentAmount as float64)+lmt.loanrequestamount)<0.295 then "<30%"
  when safe_cast(lmt.downPaymentAmount as float64)/(safe_cast(lmt.downPaymentAmount as float64)+lmt.loanrequestamount)<0.35 then "<35%"
  when safe_cast(lmt.downPaymentAmount as float64)/(safe_cast(lmt.downPaymentAmount as float64)+lmt.loanrequestamount)<0.4 then "<40%"
  else "40+" end as DP_range,
  tbtc.loanCategoryId,
  tbtc.createdDate,
  tbtc.brandName,
  tbtc.skuNo,
  tbtc.price,
  tbpm.product_id,
  tbpm.product_category,
  tbpm.product_description,
  asf.mer_category STORE_TAG,    --- This is the tag maintained by SIL team
  tbpm.product_downpayment,
  tbpm.product_crifcode,
FROM
  `risk_credit_mis.loan_master_table` lmt
LEFT JOIN
  prj-prod-dataplatform.dl_loans_db_raw.tdbk_bnpl_trans_category tbtc
ON
  lmt.digitalLoanAccountid = tbtc.digitalLoanAccountid
LEFT JOIN
  prj-prod-dataplatform.dl_loans_db_raw.tdbk_bnpl_product_mtb tbpm
ON
  tbpm.product_id = tbtc.loanCategoryId
LEFT JOIN
prj-prod-dataplatform.worktable_datachampions.AREA_SPLIT_CURRENTV2 asf
ON
 UPPER(asf.purpleKey) = UPPER(lmt.purpleKey) --
inner join `dl_loans_db_raw.tdbk_merchant_refferal_mtb` c on lmt.purpleKey=c.mer_refferal_code
 WHERE
 upper(lmt.new_loan_type) like '%SIL%'
 and lmt.loantype="BNPL"
  ;"""

In [7]:
dfd = client.query(sq).to_dataframe(progress_bar_type='tqdm')





Job ID 3f536e17-00f8-4470-b2b4-542ff092aec0 successfully executed: 100%|██████████|



Query is running:   0%|          |




























Downloading: 100%|██████████|


In [8]:
dfd.shape

(299311, 35)

In [9]:
dfd.columns

Index(['digitalLoanAccountId', 'loanAccountNumber', 'purpleKey',
       'merchantName', 'startApplyDateTime', 'applicationStatus',
       'decision_date', 'mth', 'employmentstatus', 'application_hour',
       'goods_type', 'pos_type', 'retailer', 'new_loan_type', 'loantype',
       'approved', 'disbursed', 'rejected', 'volume_out', 'tenor',
       'disbursementDateTime', 'downPaymentAmount', 'ca_category', 'DP_range',
       'loanCategoryId', 'createdDate', 'brandName', 'skuNo', 'price',
       'product_id', 'product_category', 'product_description', 'STORE_TAG',
       'product_downpayment', 'product_crifcode'],
      dtype='object')

In [10]:
dfd.head()

,digitalLoanAccountId,loanAccountNumber,purpleKey,merchantName,startApplyDateTime,applicationStatus,decision_date,mth,employmentstatus,application_hour,...,createdDate,brandName,skuNo,price,product_id,product_category,product_description,STORE_TAG,product_downpayment,product_crifcode
0,439a8f6c-0e45-4408-bf15-3a741f5fb537,None,MRON13K,FC Home Center Circuit,2023-10-29 16:11:42,RESET,NaT,None,SELF EMPLOYED,18+,...,2023-10-29 08:11:19+00:00,Astron,TC-LRE60,8859.00,1722,None,Air conditioners,Appliances Retailer,0.0,118
1,34a527e1-48b4-4398-b590-d149eda49606,None,MYXLHQ4,FC Home Center Kapasigan,2023-11-04 14:53:28,RESET,NaT,None,SALARIED,18+,...,2023-11-04 06:53:24+00:00,Megaboom,DS-91633 6 SEATER,20680.00,1726,None,"Dining set, Chairs or Tables",Appliances Retailer,0.0,122
2,af6bf9cf-dc91-428b-8f66-30f56ce1ca79,None,M69XDHR,Techsynergy Inline Liloan Grand Mall,2024-07-28 12:18:32,RESET,NaT,None,SALARIED,18+,...,2024-07-28 04:18:19+00:00,Oppo,Oppo a38 8/128gb,6999.00,1747,None,Mobile Phones,Mobile Retailer,20.0,107
3,af6bf9cf-dc91-428b-8f66-30f56ce1ca79,None,M69XDHR,Techsynergy Inline Liloan Grand Mall,2024-07-28 12:18:32,RESET,NaT,None,SALARIED,18+,...,2024-07-28 04:18:19+00:00,Mpg,Tempered,499.00,1731,None,Accessories or Peripherals,Mobile Retailer,0.0,126
4,83a65a5a-4162-4202-be61-d7610c51df68,None,M7WJX52,Home Along San Pedro,2023-06-12 14:27:21,RESET,NaT,None,SALARIED,18+,...,2023-06-12 06:27:13+00:00,Platinum,KS-5,2899.00,1720,None,"Stereo set, Soundbar, Audio equipment",Appliances Retailer,0.0,114


In [11]:
print(dfd.dtypes)

digitalLoanAccountId                 object
loanAccountNumber                    object
purpleKey                            object
merchantName                         object
startApplyDateTime           datetime64[us]
applicationStatus                    object
decision_date                datetime64[us]
mth                                  object
employmentstatus                     object
application_hour                     object
goods_type                           object
pos_type                             object
retailer                             object
new_loan_type                        object
loantype                             object
approved                              Int64
disbursed                             Int64
rejected                              Int64
volume_out                          float64
tenor                                 Int64
disbursementDateTime         datetime64[us]
downPaymentAmount                    object
ca_category                     

In [12]:
datetime_columns = ['startApplyDateTime', 'decision_date', 'disbursementDateTime', 'createdDate']

# Remove timezone information
for col in datetime_columns:
    dfd[col] = dfd[col].dt.tz_localize(None)
    


In [13]:
dfd.to_excel("Sil_DisbursedData_20241003.xlsx", index = False)